In [ ]:
# File to create full dataset with all variables based on selected entries

In [1]:
#Load packages
import pyreadr
import pandas as pd
import numpy as np
from IPython.display import display 

In [2]:
# Load created function
from utils import add_latest_entry_before_date, add_latest_diagnosis_entry

In [3]:
# Read-in dataframe generated in R
# input_data_Hb has been uploaded --> 562k entries 
input = pyreadr.read_r("/home/jupyter-niclas/input_data_Hb.rds")
input_data_Hb = next(iter(input.values()))
# Renaming of column
input_data_Hb = input_data_Hb.rename(columns={'val': 'val_HbA1c'})
# Removing column agr
input_data_Hb = input_data_Hb.drop (['cod','agr'], axis=1)
display(input_data_Hb)

,idp,dat,val_HbA1c
0,0000072333d55312fe62d5b5458a22efa4257e27,20220825,5.8
1,0000150454aa8670459408f45e453242f7bcde9c,20130121,5.8
2,0000214c9856e03de46a8f927fc1e08b7c943578,20140508,5.9
3,000021e70ca45c3421ebb2471fab947cffd3c8ff,20190111,6.3
4,000037793d3a42d2a0b2c1fd45a7ead449533aea,20130606,6.5
...,...,...,...
727288,ffff5a0589bcceac85439a31d9e7421943ed15cd,20110217,6.1
727289,ffff8531e62f5d66787f60582f9d439732650d8e,20120412,6.1
727290,ffffbdaa39d2710784eb55574a889a0ae90ab5fa,20170523,5.9
727291,ffffd90a7bcb45d93367223a6c702f04b274b8a3,20141203,5.8


In [4]:
# Adding column sexe to input_data_Hb according to data_poblacio
poblacio = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_poblacio_20240409_202620.rds")
data_poblacio = next(iter(poblacio.values()))
input_data_Hb = input_data_Hb.merge(data_poblacio[['idp', 'sexe']], on='idp', how='left')
# Check new column for number of NAs
sexe_NA = input_data_Hb['sexe'].isna().sum()
print (sexe_NA)

0


In [5]:
# Adding column birth_date to input_data_Hb according to dnaix from data_poblacio
input_data_Hb = input_data_Hb.merge(data_poblacio[['idp', 'dnaix']], on='idp', how='left')
# Check new column for number of NAs
dnaix_NA = input_data_Hb['dnaix'].isna().sum()
print (dnaix_NA)

0


In [6]:
# Using onset-date and birth date to determine age at t0
# Convert to datetime format
input_data_Hb['dat'] = pd.to_datetime(input_data_Hb['dat'].astype(str), format='%Y%m%d')
input_data_Hb['dnaix'] = pd.to_datetime(input_data_Hb['dnaix'].astype(str), format='%Y%m%d')
# Calculate age (in years)
input_data_Hb['age'] = (input_data_Hb['dat'] - input_data_Hb['dnaix']).dt.days // 365
# Exclude individuals younger than 18
input_data_Hb = input_data_Hb[input_data_Hb['age'] >= 18]
# Check for NAs in age
age_NA = input_data_Hb['age'].isna().sum()
print(f"Number of missing age entries: {age_NA}")
display (input_data_Hb)

Number of missing age entries: 0


,idp,dat,val_HbA1c,sexe,dnaix,age
0,0000072333d55312fe62d5b5458a22efa4257e27,2022-08-25,5.8,D,1949-09-01,73
1,0000150454aa8670459408f45e453242f7bcde9c,2013-01-21,5.8,H,1962-02-01,51
2,0000214c9856e03de46a8f927fc1e08b7c943578,2014-05-08,5.9,H,1947-06-01,66
3,000021e70ca45c3421ebb2471fab947cffd3c8ff,2019-01-11,6.3,H,1964-01-01,55
4,000037793d3a42d2a0b2c1fd45a7ead449533aea,2013-06-06,6.5,D,1953-03-01,60
...,...,...,...,...,...,...
727288,ffff5a0589bcceac85439a31d9e7421943ed15cd,2011-02-17,6.1,D,1944-11-01,66
727289,ffff8531e62f5d66787f60582f9d439732650d8e,2012-04-12,6.1,H,1934-03-01,78
727290,ffffbdaa39d2710784eb55574a889a0ae90ab5fa,2017-05-23,5.9,H,1948-03-01,69
727291,ffffd90a7bcb45d93367223a6c702f04b274b8a3,2014-12-03,5.8,D,1944-03-01,70


In [7]:
# Adding column smoking status from tabaquisme
tabaquisme = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_tabaquisme_20240409_202620.rds")
data_tabaquisme = next(iter(tabaquisme.values()))
# Converting code into smoking status
smoking_mapping = {
    0: "No smoker",
    1: "Smoker",
    2: "Ex-smoker"
}
data_tabaquisme['val'] = data_tabaquisme['val'].replace(smoking_mapping)
data_tabaquisme.rename(columns={'val': 'smoking_status'}, inplace=True)

In [8]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_tabaquisme,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='smoking_status'   
)

Number of missing values in 'smoking_status': 37102


In [9]:
# Adding column alcohol risk consumption from variables_cliniques --> subset ALRIS generated in R
alris = pyreadr.read_r("Data_preparation/data_ALRIS.rds")
data_alris = next(iter(alris.values()))

In [10]:
# Converting code into smoking status
alc_ris_mapping = {
    0: "No risk consumption",
    1: "Low risk consumption",
    2: "High risk consumption"
}
data_alris['val'] = data_alris['val'].replace(alc_ris_mapping)

In [11]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_alris,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='alcohol_risk_consumption'
)

Number of missing values in 'alcohol_risk_consumption': 160385


In [12]:
# Adding MEDEA index from variables_socioeconomiques
var_socioeco = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_variables_socioeconomiques_20240409_202620.rds")
data_var_socioeco = next(iter(var_socioeco.values()))
# Add both columns to dataframe
input_data_Hb = input_data_Hb.merge(data_var_socioeco[['idp', 'ruralitat', 'qmedea']], on='idp', how='left')
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_Hb['ruralitat'] = input_data_Hb['ruralitat'].replace(r'^\s*$', np.nan, regex=True)
ruralitat_NA = input_data_Hb['ruralitat'].isna().sum()
print (ruralitat_NA)
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_Hb['qmedea'] = input_data_Hb['qmedea'].replace(r'^\s*$', np.nan, regex=True)
qmedea_NA = input_data_Hb['qmedea'].isna().sum()
print (qmedea_NA)
display(input_data_Hb)

129193
248502


,idp,dat,val_HbA1c,sexe,dnaix,age,smoking_status,alcohol_risk_consumption,ruralitat,qmedea
0,0000072333d55312fe62d5b5458a22efa4257e27,2022-08-25,5.8,D,1949-09-01,73,No smoker,No risk consumption,R,NaN
1,0000150454aa8670459408f45e453242f7bcde9c,2013-01-21,5.8,H,1962-02-01,51,NaN,NaN,R,NaN
2,0000214c9856e03de46a8f927fc1e08b7c943578,2014-05-08,5.9,H,1947-06-01,66,No smoker,Low risk consumption,R,NaN
3,000021e70ca45c3421ebb2471fab947cffd3c8ff,2019-01-11,6.3,H,1964-01-01,55,Smoker,No risk consumption,U,U5
4,000037793d3a42d2a0b2c1fd45a7ead449533aea,2013-06-06,6.5,D,1953-03-01,60,Ex-smoker,Low risk consumption,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
726691,ffff5a0589bcceac85439a31d9e7421943ed15cd,2011-02-17,6.1,D,1944-11-01,66,No smoker,No risk consumption,U,U4
726692,ffff8531e62f5d66787f60582f9d439732650d8e,2012-04-12,6.1,H,1934-03-01,78,Ex-smoker,Low risk consumption,U,U4
726693,ffffbdaa39d2710784eb55574a889a0ae90ab5fa,2017-05-23,5.9,H,1948-03-01,69,Smoker,No risk consumption,R,NaN
726694,ffffd90a7bcb45d93367223a6c702f04b274b8a3,2014-12-03,5.8,D,1944-03-01,70,No smoker,No risk consumption,U,U2


In [13]:
# Start Block Comorbidities
# creating dataframe data_diagnostics
diagnostics = pyreadr.read_r("Data_preparation/data_both_diagnostics.rds")
data_diagnostics = next(iter(diagnostics.values()))

In [14]:
#hypertension
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I10', 'I11', 'I12', 'I13', 'I14', 'I15'],
    reference_date_col='dat',
    result_col_name='hypertension'
)

Number of missing values in 'hypertension': 547912


In [15]:
# dyslipidemia
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['E78'],
    reference_date_col='dat',
    result_col_name='dyslipidemia'
)

Number of missing values in 'dyslipidemia': 559326


In [16]:
# Heart failure
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I50'],
    reference_date_col='dat',
    result_col_name='heart_failure'
)

Number of missing values in 'heart_failure': 696400


In [17]:
# Peripheral artery disease
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    fixed_codes=['I73.9'],
    reference_date_col='dat',
    result_col_name='peripheral_artey_disease'
)

Number of missing values in 'peripheral_artey_disease': 711782


In [18]:
# stroke
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I61', 'I62', 'I63'],
    reference_date_col='dat',
    result_col_name='stroke'
)

Number of missing values in 'stroke': 697623


In [19]:
# ischemic heart disease
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I20', 'I21', 'I22', 'I23','I24', 'I25'],
    reference_date_col='dat',
    result_col_name='ischemic_heart_disease'
)

Number of missing values in 'ischemic_heart_disease': 681814


In [20]:
# Chronic kidney disease
input_data_Hb = add_latest_diagnosis_entry(
    goal_df=input_data_Hb,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['N18', 'I12', 'I13'],
    reference_date_col='dat',
    result_col_name='chronic_kidney_disease'
)

Number of missing values in 'chronic_kidney_disease': 678455


In [21]:
#END section comorbidities
display(input_data_Hb)

,idp,dat,val_HbA1c,sexe,dnaix,age,smoking_status,alcohol_risk_consumption,ruralitat,qmedea,hypertension,dyslipidemia,heart_failure,peripheral_artey_disease,stroke,ischemic_heart_disease,chronic_kidney_disease
0,0000072333d55312fe62d5b5458a22efa4257e27,2022-08-25,5.8,D,1949-09-01,73,No smoker,No risk consumption,R,NaN,1,1,NaN,NaN,NaN,NaN,NaN
1,0000150454aa8670459408f45e453242f7bcde9c,2013-01-21,5.8,H,1962-02-01,51,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2,0000214c9856e03de46a8f927fc1e08b7c943578,2014-05-08,5.9,H,1947-06-01,66,No smoker,Low risk consumption,R,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3,000021e70ca45c3421ebb2471fab947cffd3c8ff,2019-01-11,6.3,H,1964-01-01,55,Smoker,No risk consumption,U,U5,1,NaN,NaN,NaN,NaN,NaN,NaN
4,000037793d3a42d2a0b2c1fd45a7ead449533aea,2013-06-06,6.5,D,1953-03-01,60,Ex-smoker,Low risk consumption,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726691,ffff5a0589bcceac85439a31d9e7421943ed15cd,2011-02-17,6.1,D,1944-11-01,66,No smoker,No risk consumption,U,U4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726692,ffff8531e62f5d66787f60582f9d439732650d8e,2012-04-12,6.1,H,1934-03-01,78,Ex-smoker,Low risk consumption,U,U4,1,NaN,NaN,NaN,NaN,NaN,NaN
726693,ffffbdaa39d2710784eb55574a889a0ae90ab5fa,2017-05-23,5.9,H,1948-03-01,69,Smoker,No risk consumption,R,NaN,1,1,NaN,NaN,NaN,NaN,NaN
726694,ffffd90a7bcb45d93367223a6c702f04b274b8a3,2014-12-03,5.8,D,1944-03-01,70,No smoker,No risk consumption,U,U2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Adding column BMI from variables cliniques --> subset BMI generated in R
BMI = pyreadr.read_r("Data_preparation/data_BMI.rds")
data_BMI = next(iter(BMI.values()))

In [23]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_BMI,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='BMI'
)

Filtered out 0 negative values in 'BMI'.
Number of missing values in 'BMI': 171206


In [24]:
# Adding column Systolic blood pressure from variables cliniques --> subset SysBP generated in R
SysBP = pyreadr.read_r("Data_preparation/data_SysBP.rds")
data_SysBP = next(iter(SysBP.values()))

In [25]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_SysBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='systolic_blood_pressure'
)

Filtered out 0 negative values in 'systolic_blood_pressure'.
Number of missing values in 'systolic_blood_pressure': 86351


In [26]:
# Adding column Diastolic blood pressure from variables cliniques --> subset DiasBP generated in R
DiasBP = pyreadr.read_r("Data_preparation/data_DiasBP.rds")
data_DiasBP = next(iter(DiasBP.values()))

In [27]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_DiasBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='diastolic_blood_pressure'
)

Filtered out 0 negative values in 'diastolic_blood_pressure'.
Number of missing values in 'diastolic_blood_pressure': 86330


In [28]:
# Adding column cholesterol LDL from variables analitiques --> subset cLDL generated in R
cLDL = pyreadr.read_r("Data_preparation/data_cLDL.rds")
data_cLDL = next(iter(cLDL.values()))

In [29]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_cLDL,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='cholesterol_LDL'
)

Filtered out 0 negative values in 'cholesterol_LDL'.
Number of missing values in 'cholesterol_LDL': 208379


In [30]:
# Adding column triglycerides from variables analitiques --> subset trigly generated in R
trigly = pyreadr.read_r("Data_preparation/data_trigly.rds")
data_trigly = next(iter(trigly.values()))

In [31]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_trigly,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='triglycerides'
)

Filtered out 899 negative values in 'triglycerides'.
Number of missing values in 'triglycerides': 168210


In [32]:
# Adding column eGFR from variables analitiques --> subset eGFR generated in R
eGFR = pyreadr.read_r("Data_preparation/data_eGFR.rds")
data_eGFR = next(iter(eGFR.values()))

In [33]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_eGFR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='eGFR'
)

Filtered out 0 negative values in 'eGFR'.
Number of missing values in 'eGFR': 161875


In [34]:
# Adding column ACR from variables cliniques --> subset ACR generated in R
ACR = pyreadr.read_r("Data_preparation/data_ACR.rds")
data_ACR = next(iter(ACR.values()))

In [35]:
input_data_Hb = add_latest_entry_before_date(
    goal_df=input_data_Hb,
    data_df=data_ACR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='ACR'
)

Filtered out 18550 negative values in 'ACR'.
Number of missing values in 'ACR': 455795


In [36]:
display(input_data_Hb)

,idp,dat,val_HbA1c,sexe,dnaix,age,smoking_status,alcohol_risk_consumption,ruralitat,qmedea,...,stroke,ischemic_heart_disease,chronic_kidney_disease,BMI,systolic_blood_pressure,diastolic_blood_pressure,cholesterol_LDL,triglycerides,eGFR,ACR
0,0000072333d55312fe62d5b5458a22efa4257e27,2022-08-25,5.8,D,1949-09-01,73,No smoker,No risk consumption,R,NaN,...,NaN,NaN,NaN,25.78,126.0,68.0,103.0,82.0,60.1,5.58
1,0000150454aa8670459408f45e453242f7bcde9c,2013-01-21,5.8,H,1962-02-01,51,NaN,NaN,R,NaN,...,1,NaN,NaN,NaN,113.0,70.0,NaN,183.0,NaN,NaN
2,0000214c9856e03de46a8f927fc1e08b7c943578,2014-05-08,5.9,H,1947-06-01,66,No smoker,Low risk consumption,R,NaN,...,NaN,1,NaN,24.02,135.0,75.0,103.0,128.0,60.1,NaN
3,000021e70ca45c3421ebb2471fab947cffd3c8ff,2019-01-11,6.3,H,1964-01-01,55,Smoker,No risk consumption,U,U5,...,NaN,NaN,NaN,29.27,156.0,87.0,101.0,169.0,60.1,10.00
4,000037793d3a42d2a0b2c1fd45a7ead449533aea,2013-06-06,6.5,D,1953-03-01,60,Ex-smoker,Low risk consumption,NaN,NaN,...,NaN,NaN,NaN,29.58,120.0,70.0,155.0,385.0,60.1,8.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726691,ffff5a0589bcceac85439a31d9e7421943ed15cd,2011-02-17,6.1,D,1944-11-01,66,No smoker,No risk consumption,U,U4,...,NaN,NaN,NaN,39.56,130.0,68.0,NaN,NaN,NaN,NaN
726692,ffff8531e62f5d66787f60582f9d439732650d8e,2012-04-12,6.1,H,1934-03-01,78,Ex-smoker,Low risk consumption,U,U4,...,NaN,NaN,NaN,27.43,122.0,68.0,93.0,152.0,60.1,44.20
726693,ffffbdaa39d2710784eb55574a889a0ae90ab5fa,2017-05-23,5.9,H,1948-03-01,69,Smoker,No risk consumption,R,NaN,...,NaN,NaN,NaN,27.31,132.0,81.0,105.0,122.0,60.1,NaN
726694,ffffd90a7bcb45d93367223a6c702f04b274b8a3,2014-12-03,5.8,D,1944-03-01,70,No smoker,No risk consumption,U,U2,...,NaN,NaN,NaN,33.05,142.0,74.0,159.0,111.0,60.1,NaN


In [37]:
# Save generated input_data_Hb-file
input_data_Hb.to_csv("generated_input_data_Hb.csv", index=False)